# 第 3 章 让大模型使用各种工具
经常使用大模型的你，一定会发现大模型并不能回答所有的问题。比如大模型大概率无法回答：你的同事小明是谁，如果小明不是特别出名的话。

本章将通过一个简单的例子，让你快速了解如何让大模型利用外部工具，来回答一些原本无法回答的问题。


## 1. 准备工作

### 1.1. 安装

下载文档代码及安装依赖项
```bash
git clone https://github.com/AlibabaCloudDocs/llm_learning.git
cd llm_learning
pip install -r requirements.txt
```

### 1.2. 账号准备

首先，您需要前往 [官网创建 API Key](https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key)。接下来，请获取你的 [DASHSCOPE_API_KEY](https://dashscope.console.aliyun.com/apiKey)

####  MacOS or Linux
您可以使用以下命令行导入环境变量
```bash
export DASHSCOPE_API_KEY="sk-****"
```

#### Windows
可以在终端使用[`SET`](https://learn.microsoft.com/zh-cn/windows-server/administration/windows-commands/set_1)命令设置环境变量
```bat
set DASHSCOPE_API_KEY=sk-****
```
或者在[`PowerShell`](https://learn.microsoft.com/zh-cn/powershell/module/microsoft.powershell.core/about/about_environment_variables?view=powershell-7.4)中使用以下命令行配置环境变量 
```powershell
$Env:DASHSCOPE_API_KEY = "sk-****"
```

#### Jupyter Notebook
您可以使用[`os.environ`](https://docs.python.org/3/library/os.html)方法，在代码开头设置临时环境变量。

### 1.3. docenv模式

将环境变量写入文件"~/.zshrc"中：

```bash
export OPENAI_API_KEY="sk-..."
```
就可以执行如下命令 ```source ~/.zshrc``` 将这个环境变量绑定到全局shell中。

接下来我们将加载这个环境变量到notebook中。执行如下命令

**使用Windows，已经通过Windows PowerShell来注册环境变量的同事可以考虑跳过这里**

In [1]:
import os
from dotenv import load_dotenv
## for MacOS users
filePath = os.path.abspath(os.path.expanduser(os.path.expandvars("~/.zshrc")))
load_dotenv(filePath)

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8
Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 16


True

## 2.快速开始

### 2.1.与Qwen-MAX直接对话


首先我们尝试直接询问大模型关于“阿里云灵积服务”是什么的问题，方式如下：

In [2]:
from langchain_community.llms import Tongyi

llm = Tongyi()
# 使用更强大的通义千问 max
llm.model_name = 'qwen-max'

print(llm.invoke('灵积是什么服务'))

非常抱歉，根据当前可获取的信息和知识库，未能找到关于“灵积”这一具体服务的详细描述或相关资料。可能有以下几种情况：

1. **名称不准确或拼写错误**：请核对您提供的服务名称是否正确，是否存在笔误或缩写导致的信息不完整。如有必要，请提供更详细的信息、全称或相关上下文，以便我为您提供准确解答。

2. **新兴或本地化服务**：“灵积”可能是某个新兴的、尚未广泛普及的服务，或者是一个特定地区或行业内的专属服务，因此在公开资源中难以查询到详细信息。如果可以，请提供更多背景信息或服务特点，我将尽力根据这些线索为您推测其可能的含义或性质。

3. **企业内部服务或项目代号**：“灵积”也有可能是某个公司、组织内部使用的特定服务、项目或产品的名称，这类信息通常不在公开渠道披露。如果您是在企业内部环境中接触到这个名词，建议咨询相关同事或查阅内部文档以获取准确信息。

鉴于以上情况，建议您直接向提供该服务的机构、公司或相关人员咨询，以获得最准确、详尽的信息。如您能提供更多的背景信息或详细描述，我很愿意进一步帮您分析和理解“灵积”可能指代的服务内容。


我们可以看到由于大模型不了解这个服务，所以在创造一个它认为合理的答案。

接下来我们来为大模型添加一些知识，让他能正确回答这个问题。


### 2.2.基于Langchain调用自定义工具

我们首先通过调用langchain来实现自定义工具的调用，然后我们将手动实现langchain的能力，从而让大家很好的理解其中的奥妙。

这里我们将使用langchain中的AgentExecutor来实现这个能力：

（注意：如果你遇到了 TypeError: Additional kwargs key output_tokens already exists in left dict and value has unsupported type <class 'int'> 这个错误，请参考[github](https://github.com/langchain-ai/langchain/pull/16580)中的说明）

In [3]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
from langchain_community.llms import Tongyi
from langchain_core.tools import BaseTool

model = Tongyi()
model.model_name = 'qwen-max'


class SearchTool(BaseTool):
    """服务查询工具"""

    name: str = "服务查询工具"
    description: str = (
        "当你不确定一个服务是什么，才使用此工具。"
    )

    def _run(self, name: str) -> str:
        if name == '灵积':
            return '''DashScope灵积模型服务建立在“模型即服务”（Model-as-a-Service，MaaS）的理念基础之上，围绕AI各领域模型，通过标准化的API提供包括模型推理、模型微调训练在内的多种模型服务。
DashScope灵积模型服务依托于业界各领域的优质模型，基于阿里云强大的基础设施搭建。欢迎AI应用开发者由此开启模型探索之旅！'''
        return name + '抱歉，没有查到相关信息。'


tools = [SearchTool()]

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({'input': '灵积是什么服务'})
print(result['output'])




> Entering new AgentExecutor chain...
 我需要查询灵积这个服务的详细信息
Action: 服务查询工具
Action Input: 灵积DashScope灵积模型服务建立在“模型即服务”（Model-as-a-Service，MaaS）的理念基础之上，围绕AI各领域模型，通过标准化的API提供包括模型推理、模型微调训练在内的多种模型服务。
DashScope灵积模型服务依托于业界各领域的优质模型，基于阿里云强大的基础设施搭建。欢迎AI应用开发者由此开启模型探索之旅！ 我已经了解了灵积服务的基本信息
Final Answer: 灵积是基于“模型即服务”（Model-as-a-Service，MaaS）理念的一款服务产品，它汇聚了AI各领域的优质模型资源，通过标准化API为用户提供诸如模型推理、模型微调训练等多种模型服务。该服务由阿里云强大的基础设施支持，旨在为AI应用开发者提供便捷、高效的模型探索与应用平台。

> Finished chain.
灵积是基于“模型即服务”（Model-as-a-Service，MaaS）理念的一款服务产品，它汇聚了AI各领域的优质模型资源，通过标准化API为用户提供诸如模型推理、模型微调训练等多种模型服务。该服务由阿里云强大的基础设施支持，旨在为AI应用开发者提供便捷、高效的模型探索与应用平台。


## 3.原理说明

第一次执行时，我们直接向大模型提问：灵积是什么服务。 这时大模型并没有在预训练中学习过关于“灵积”服务的知识，因此无法回答。

为了解决这个问题，我们可以借助外部工具，比如搜索引擎工具如Elastic Search等，先查询一下灵积是什么服务，然后再向大模型提问。有了准确的信息补充，大模型就能回答的更准确了。

上述代码采用了"hwchase17/react"风格，这种prompt风格定义为：
```
input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] 
template='Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'
```

在上述prompt中，大模型根据输入的问题{question}，来构造了一个思维链。在这个思维链中：问题、思考、行为、行动的输入、行动的结果，这五点是Agent工作的核心。在思考中，我们定义了Agent的scratchpad，这个scratchpad可以用于记录Agent的思考过程。在行动中，我们定义了Agent的工具{tools}，这个工具可以用于帮助Agent解决问题。这个思考过程可以进行N次直到Agent认为自己已经解决了问题。这时通过Thought: I now know the final answer来结束这个思考过程，然后通过Final Answer来输出最终的答案。

理解了上述过程，我们就可以考虑如何实现Agent的工具。

## 4.自定义大模型XML交互工具

### 4.1.定义XML交互格式

基于上述Langchain的思路我们来研究如何自定义一个工具。为了便于和大模型交互，我们定义一个XML交互工具。当用户向大模型提问时，大模型必须返回XML格式的回复，便于我们后续解析和调用工具。


In [6]:
from langchain_community.llms import Tongyi

llm = Tongyi()
# 使用更强大的通义千问 max
llm.model_name = 'qwen-max'

prompt = '''你是一个可以回答任何问题的助手。
你可以使用下列工具: 

find_service: 当你不确定答案时，你可以使用此工具。

为了使用这个工具，你必须用<tool></tool>和<tool_input></tool_input>标签。
例如，如果您有一个名为“find_service”的工具，可以查询企业内部服务信息，为了搜索阿里云是什么服务，你会返回：
<tool>find_service</tool><tool_input>阿里云</tool_input>

使用工具后你会得到一个形式为<observation></observation>的响应，因此在第二轮你会得到输入为
<tool>find_service</tool><tool_input>阿里云</tool_input><observation>阿里云服务简介...</observation>

你需要判断<observation></observation>标签中的内容是不是你需要的答案。如果是最终答案final_answer，你需要返回答案：
<final_answer>阿里云服务简介...</final_answer>

如果你不需要使用工具也能回答问题，你也必须要用<final_answer></final_answer>标签来包裹答案。比如
<final_answer>阿里云服务详细介绍...</final_answer>

开始任务：

问题：
'''

print(llm.invoke(prompt + '灵积是什么服务'))

<tool>find_service</tool><tool_input>灵积</tool_input>


### 4.2.定义XML解析函数和搜索工具

In [7]:
from langchain.output_parsers import XMLOutputParser

# 从xml中获取tool和tool_input
def get_tool_and_input(response):
    xml_response = f"<xml>{response}</xml>"
    parsed_xml_response = XMLOutputParser().invoke(xml_response)
    response_dict = {}
    for item in parsed_xml_response['xml']:
        for key in item:
            response_dict[key] = item[key]
    return response_dict

# 运行tool
def run_tool(response_dict):
    search_input =''
    search_result = ''
    if response_dict.get('tool') is None:
        search_result = response
    elif response_dict['tool'] == 'find_service':
        search_input = response_dict['tool_input']
        search_result = SearchTool().run(search_input)
    return search_input, search_result

response="""<tool>find_service</tool><tool_input>灵积</tool_input>"""
response_dict = get_tool_and_input(response)
search_input, search_result = run_tool(response_dict)
search_result


'DashScope灵积模型服务建立在“模型即服务”（Model-as-a-Service，MaaS）的理念基础之上，围绕AI各领域模型，通过标准化的API提供包括模型推理、模型微调训练在内的多种模型服务。\nDashScope灵积模型服务依托于业界各领域的优质模型，基于阿里云强大的基础设施搭建。欢迎AI应用开发者由此开启模型探索之旅！'

### 4.3.一个完整的工具定义

为了更好帮助读者理解大模型Agent调用的原理，下面的代码是我们自己定义了一个简单的工具类 SearchTool 类，而不是继承自langchain的 BaseTool。并且我们实现了一个简单的 DIYAgent 类，这个类参考了langchain的接口，实现基本的调用工具的能力。这样读者可以看到大模型是如何理解用户问题并返回调用工具的指令，而Agent又是如何理解这个指令，并启动对应工具的。

In [15]:
from langchain_community.llms import Tongyi
import json

# 定义模型
llm = Tongyi()
llm.model_name = 'qwen-max'

# 定义搜索工具
class SearchTool():
    """服务查询工具"""

    name: str = "服务查询工具"
    description: str = (
        "当你不确定一个服务是什么，才使用此工具。"
    )

    def run(self, name: str) -> str:
        if name == '灵积':
            return '''DashScope灵积模型服务建立在“模型即服务”（Model-as-a-Service，MaaS）的理念基础之上，围绕AI各领域模型，通过标准化的API提供包括模型推理、模型微调训练在内的多种模型服务。DashScope灵积模型服务依托于业界各领域的优质模型，基于阿里云强大的基础设施搭建。欢迎AI应用开发者由此开启模型探索之旅！'''
        return name + '抱歉，没有查到相关信息。'
    

# 定义查询模版
prompt_template = '''你是一个可以回答任何问题的助手。
你可以使用下列工具: 

{tools}: 当你不确定答案时，你可以使用此工具。

为了使用这个工具，你必须用<tool></tool>和<tool_input></tool_input>标签。
例如，如果您有一个名为“{tool1}”的工具，可以查询企业内部服务信息，为了搜索(阿里云)是什么服务，你会返回：
<tool>{tool1}</tool><tool_input>阿里云</tool_input>

使用工具后你会得到一个形式为<observation></observation>的响应，因此在第二轮你会得到输入为
<tool>{tool1}</tool><tool_input>阿里云</tool_input><observation>阿里云服务简介XXX</observation>

你需要判断<observation></observation>标签中的内容是不是你需要的答案。如果是最终答案final_answer，你需要返回答案：
<final_answer>阿里云服务简介XXX</final_answer>

如果你不需要使用工具也能回答问题，你也必须要用<final_answer></final_answer>标签来包裹答案。比如
<final_answer>阿里云服务详细介绍XXX</final_answer>

开始任务：

问题：
'''


# 定义一个简单的工具调用Agent
class DIYAgent():
    def __init__(self,tool,model=None,prompt=None):
        self.tool = tool
        self.model = model
        self.verbose = False
        self.tool_name = self.get_class_name(self.tool) 
        self.prompt = prompt.format(tools=self.tool_name,tool1 = self.tool_name)

    def get_class_name(self,tool):
        return tool.__class__.__name__
    
    # 从xml中获取tool和tool_input
    def get_tool_and_input(self, response):
        xml_response = f"<xml>{response}</xml>"
        parsed_xml_response = XMLOutputParser().invoke(xml_response)
        response_dict = {}
        for item in parsed_xml_response['xml']:
            for key in item:
                response_dict[key] = item[key]
        return response_dict

    # 执行工具
    def run_tool(self, response_dict):
        search_input =''
        search_result = ''

        if response_dict.get('tool') is None:
            if response_dict.get('final_answer') is not None:
                return search_input, response_dict['final_answer']
            else:
                return search_input, "Error in tool"+json.dumps(response_dict)

        if response_dict['tool'] ==  self.tool_name:
            search_input = response_dict['tool_input']
            search_result = self.tool.run(search_input)
            return search_input, search_result

    # run agent
    def run(self, request):        
        raw_response = self.model.invoke(self.prompt + request)
        print("原始响应：", raw_response)
        response_dict = self.get_tool_and_input(raw_response)
        print("使用工具：", response_dict['tool'])
        print("查询内容：", response_dict['tool_input'])
        search_input, search_result = self.run_tool(response_dict)
        print("搜索结果：", search_result)

        new_prompt = self.prompt + f"<tool>{self.tool_name}</tool><tool_input>{search_input}</tool_input><observation>{search_result}</observation>"
        final_response = self.model.invoke(new_prompt)
        final_response_dict = self.get_tool_and_input(final_response)
        search_input, final_answer = self.run_tool(final_response_dict)
        return final_answer


# MIAN Function
tool = SearchTool()

diyAgent = DIYAgent(tool ,llm, prompt_template)

result = diyAgent.run("灵积是什么服务")
print("最终输出：",result)


原始响应： <tool>SearchTool</tool><tool_input>灵积 服务</tool_input>
使用工具： SearchTool
查询内容： 灵积 服务
搜索结果： 灵积 服务抱歉，没有查到相关信息。
最终输出： 很抱歉，根据当前提供的信息源，未能找到关于“灵积 服务”的相关资料。这意味着该服务可能不存在，或者其名称有误，或者相关信息不在可用的数据库或搜索引擎范围内。如果您能提供更详细、准确的服务名称、所属领域、相关机构等信息，我将更有可能帮助您找到有关该服务的确切资料。如您有其他问题或需要查询其他服务信息，请随时告知。


### 4.4.添加更多的搜索词条

扩展搜索范围时，我们只需要扩展SearchTool函数的能力即可，比如从搜索引擎上下载信息。为了进行对比，我们直接填入一些已知信息。


In [16]:
# 扩展支持的服务查询
class SearchTool2():
    """服务查询工具"""

    name: str = "服务查询工具"
    description: str = (
        "当你不确定一个服务是什么，才使用此工具。"
    )

    def run(self, name: str) -> str:
        if name == '灵积':
            return '''DashScope灵积模型服务建立在“模型即服务”（Model-as-a-Service，MaaS）的理念基础之上，围绕AI各领域模型，通过标准化的API提供包括模型推理、模型微调训练在内的多种模型服务。DashScope灵积模型服务依托于业界各领域的优质模型，基于阿里云强大的基础设施搭建。欢迎AI应用开发者由此开启模型探索之旅！'''
        elif name=="百炼":
            return "百炼，即大模型服务平台，是面向企业客户及合作伙伴的，基于通义大模型、行业大模型以及三方大模型，结合企业专属数据，包含全链路大模型开发工具的一站式大模型商业化平台。提供完整的模型训练、微调、评估等产品工具，预置丰富的应用插件，提供便捷的集成方式，更快更高效地完成大模型应用的构建。"
        elif (name=="PAI") or (name=="阿里云 PAI"):
            return "人工智能平台 PAI（Platform of Artificial Intelligence）面向企业客户及开发者，提供轻量化、高性价比的云原生人工智能，涵盖DSW交互式建模、Designer拖拽式可视化建模、DLC分布式训练到EAS模型在线部署的全流程。"
        elif name=="OSS":
            return "阿里云对象存储OSS（Object Storage Service）是一款海量、安全、低成本、高可靠的云存储服务，可提供99.9999999999%（12个9）的数据持久性，99.995%的数据可用性。多种存储类型供选择，全面优化存储成本。"
        return name + '抱歉，没有查到相关信息。'
    

# MIAN Function
tool = SearchTool2()

diyAgent = DIYAgent(tool ,llm, prompt_template)

result = diyAgent.run("百炼是什么服务")
print("最终输出：",result)


原始响应： <tool>SearchTool2</tool><tool_input>百炼</tool_input>
使用工具： SearchTool2
查询内容： 百炼
搜索结果： 百炼，即大模型服务平台，是面向企业客户及合作伙伴的，基于通义大模型、行业大模型以及三方大模型，结合企业专属数据，包含全链路大模型开发工具的一站式大模型商业化平台。提供完整的模型训练、微调、评估等产品工具，预置丰富的应用插件，提供便捷的集成方式，更快更高效地完成大模型应用的构建。
最终输出： 百炼是面向企业客户及合作伙伴的大模型服务平台，基于通义大模型、行业大模型以及三方大模型，结合企业专属数据，提供全链路大模型开发工具，包括模型训练、微调、评估等产品工具。该平台预置丰富应用插件，支持便捷的集成方式，旨在帮助用户快速高效地构建大模型应用。


### 4.5.使用封装好的代码来实现服务查询

我们将上述代码重构后封装在文件[`search-with-tool.py`](search-with-tool.py)中。为了便于后续章节扩展，我们实现了列表式工具加载，与langchain的接口保持一致。
  
列表式工具加载：```tool = [SearchTool()]```

你可以尝试用自己的方式来修改代码。这段代码可以用以下方式来运行：

```bash
python3 search-with-tool.py -question="the question"
```

In [17]:
! python search-with-tool.py --question="百炼是什么服务"

正在分析内容...
使用工具：SearchTool...
查询内容：百炼...
查询到参考信息：百炼，即大模型服务平台，是面向企业客户及合作伙伴的，基于通义...
正在分析内容...
最终答案：百炼是面向企业客户及合作伙伴的大模型服务平台，基于通义大模型、行业大模型以及三方大模型，结合企业专属数据，具备全链路大模型开发工具，支持一站式大模型商业化应用。平台提供模型训练、微调、评估等全套产品工具，内置丰富应用插件，并以便捷的集成方式助力用户快速高效构建大模型应用。


In [18]:
! python search-with-tool.py --question="PAI是什么服务"

正在分析内容...
使用工具：SearchTool...
查询内容：PAI...
查询到参考信息：人工智能平台 PAI（Platform of Artific...
正在分析内容...
最终答案：PAI（Platform of Artificial Intelligence）是面向企业客户及开发者的人工智能平台，它提供轻量化、高性价比的云原生人工智能服务。PAI覆盖了从DSW交互式建模、Designer拖拽式可视化建模、DLC分布式训练到EAS模型在线部署的全流程。


## 5. 总结
本章我们首先用LangChain的的接口定义了一个Agent应用，为了理解大模型使用Agent的内核，我们用自己的代码实现了一遍。这段代码包括工具定义类prompt定义模版，工具调用，工具查询等功能。从这里我们看到，LangChain方法主要思路是通过详细定义的Prompt让大模型反复思考“大模型目前获得的信息是否可以回答用户的问题”，如果不能解答用户的问题，大模型应该使用何种工具；如果可以解答用户的问题，那么就输出结果。
基于以上分析，为了实现更强大的大模型助理的能力，我们还可能需要为大模型加载更多功能。比如加入一些记忆能力让大模型可以把查询结果、历史对话、历史运行结果进行缓存，这样大模型有可能知道他的目标是什么，现在是什么进度，接下来可以怎么做。这样我们还需要为大模型加入规划的能力，大模型就可以在某一个阶段里，更好的规划下一步行动的计划。我们还可以为大模型加入更多外界反馈，甚至加入机械手和视觉系统，这样大模型就可能做到指导一台机器人去为人类打一杯咖啡。但是，接下来我们想先走一小步，让大模型写代码。

## 6. 参考资料
- [DashScope](https://dashscope.aliyun.com/)
- [LangChain](https://python.langchain.com/docs)